# Iteration 0: Simple model

Pour un premier model on va prendre tout simplement réseau neuronal assez simple en utilisant comme feature d'entrées les données d'un mel spectrogramme de 32 x 32.

In [140]:
#Let's first import the module that we will need
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

In [39]:
def get_df_melspec(type_machinery):
    
    #Constants
    PATH_FEATURES_FOLDER = './Features/'
    PATH_MELSPEC_32_32_FOLDER = PATH_FEATURES_FOLDER + 'melspec_32_32/'
    PATH_MELSPEC_MACHINE_FOLDER = PATH_MELSPEC_32_32_FOLDER + type_machinery + '/'

    #Get all files in folder
    files = [f for f in os.listdir(PATH_MELSPEC_MACHINE_FOLDER)]
    
    #List used for creating the df
    list_filepath = []
    list_filename = []
    list_section = []
    list_domain_env = []
    list_dir = []
    list_sound_type = []
    list_id = []
    list_suffix = []
    list_flattened_melspec = []

    #Loop through files
    for filename in files:

        splitted_filename = filename.split('_')

        list_filepath.append(PATH_MELSPEC_MACHINE_FOLDER + filename)
        list_filename.append(filename)
        list_section.append(int(splitted_filename[1]))
        list_domain_env.append(splitted_filename[2])
        list_dir.append(splitted_filename[3])
        list_sound_type.append(splitted_filename[4])
        list_id.append(splitted_filename[5])
        list_suffix.append('_'.join(splitted_filename[6:]).split('.cs')[0])
        list_flattened_melspec.append(
            np.genfromtxt(PATH_MELSPEC_MACHINE_FOLDER + filename, delimiter=';').reshape(1,1024)
        )

    #Create dictionnary that will be transformed in df
    dict_metada = {
        'filepath' : list_filepath,
        'filename' : list_filename,
        'section' : list_section,
        'domain_env' : list_domain_env,
        'dir' : list_dir,
        'sound_type' : list_sound_type,
        'id' : list_id,
        'suffix' : list_suffix
    }

    #Get dataframe
    df = pd.DataFrame(dict_metada)
    df = pd.concat([df, pd.DataFrame(np.row_stack(list_flattened_melspec))], axis=1)
    return df
    

## Valve

In [40]:
df = get_df_melspec('valve')
df.head()

,filepath,filename,section,domain_env,dir,sound_type,id,suffix,0,1,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,./Features/melspec_32_32/valve/section_00_sour...,section_00_source_test_anomaly_0000_pat_00.csv,0,source,test,anomaly,0000,pat_00,-585.808350,-675.984131,...,-1236.630371,-1131.075073,-1024.507812,-1313.956787,-1460.510864,-1512.475464,-1606.135254,-1687.531738,-1785.416016,-1906.613770
1,./Features/melspec_32_32/valve/section_00_sour...,section_00_source_test_anomaly_0001_pat_00.csv,0,source,test,anomaly,0001,pat_00,-415.437073,-303.174377,...,-2041.975098,-2113.747559,-2140.543457,-2211.894043,-2297.982178,-2354.358154,-2368.700928,-2425.513672,-2538.235107,-2694.231689
2,./Features/melspec_32_32/valve/section_00_sour...,section_00_source_test_anomaly_0002_pat_01.csv,0,source,test,anomaly,0002,pat_01,-426.344360,-623.100403,...,-1102.953369,-1127.849243,-1160.414062,-1233.081543,-1343.235596,-1354.572021,-1397.549072,-1380.051392,-1556.630127,-1756.758301
3,./Features/melspec_32_32/valve/section_00_sour...,section_00_source_test_anomaly_0003_pat_00.csv,0,source,test,anomaly,0003,pat_00,-840.777588,-778.587158,...,-1346.578247,-1414.636719,-1524.184326,-1643.187256,-1697.765991,-1713.408936,-1645.417969,-1680.654541,-1689.903076,-1769.859131
4,./Features/melspec_32_32/valve/section_00_sour...,section_00_source_test_anomaly_0004_pat_01.csv,0,source,test,anomaly,0004,pat_01,-379.820923,-560.995117,...,-1703.000732,-1777.026367,-1840.749268,-1928.084351,-2019.248779,-2063.729248,-2186.402832,-2324.577637,-2471.290283,-2595.243164


### ANN: 1024 -> 128 -> 128 -> 64 -> normal/anormal

First let's get the data we will use to train the model

In [77]:
#Encode suffix label
label_encoder = LabelEncoder()
df['suffix_as_int'] = label_encoder.fit_transform(df['suffix'])

#Take 150 normal rows section 0
normal_sect_0 = df[((df['section'] == 0 ) & (df['sound_type'] == 'normal' ))].sample(n=150, weights='suffix_as_int')

#Take 150 normal rows section 1
normal_sect_1 = df[((df['section'] == 1 ) & (df['sound_type'] == 'normal' ))].sample(n=150, weights='suffix_as_int')

#Take 150 normal rows section 2
normal_sect_2 = df[((df['section'] == 2 ) & (df['sound_type'] == 'normal' ))].sample(n=150, weights='suffix_as_int')

#Take the 300 anormal rows (100 for each section)
anormal = df[df['sound_type'] == 'anomaly']

#Concat together
data = pd.concat([normal_sect_0, normal_sect_1, normal_sect_2, anormal])

Get train and test dataset 

In [129]:
#Encode normal label to get y
y = data['sound_type'].replace(['normal', 'anomaly'], [0, 1])

#Get X
X = data.drop(
    ['filepath', 'filename', 'section', 'domain_env', 'dir', 'sound_type', 'id', 'suffix', 'suffix_as_int'],
    axis=1
)

#Get train and test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#Scale data
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

Initialization of the ANN

In [130]:
#Initializaiton of the layers
inputs = Input(shape=X_train.shape[1], name='Inputs')
first_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='First_layer')
second_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='Second_layer')
third_layer = Dense(64, activation='relu', kernel_initializer ='normal', name='Third_layer')
fourth_layer = Dense(1, activation='sigmoid', kernel_initializer ='normal', name='Output_layer')

#Construct the layer's order
x = first_layer(inputs)
x = second_layer(x)
x = third_layer(x)
outputs = fourth_layer(x)

#Initialize the model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Inputs (InputLayer)         [(None, 1024)]            0         
                                                                 
 First_layer (Dense)         (None, 128)               131200    
                                                                 
 Second_layer (Dense)        (None, 128)               16512     
                                                                 
 Third_layer (Dense)         (None, 64)                8256      
                                                                 
 Output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 156,033
Trainable params: 156,033
Non-trainable params: 0
_________________________________________________________________


Compile the model

In [132]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Let's train our model!

In [133]:
model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)

Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.6957 - accuracy: 0.5333 - val_loss: 0.7225 - val_accuracy: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6723 - accuracy: 0.5938 - val_loss: 0.6914 - val_accuracy: 0.6250
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6714 - accuracy: 0.5938 - val_loss: 0.7035 - val_accuracy: 0.6167
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6566 - accuracy: 0.6000 - val_loss: 0.7146 - val_accuracy: 0.5917
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6474 - accuracy: 0.6438 - val_loss: 0.6886 - val_accuracy: 0.5917
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6325 - accuracy: 0.6354 - val_loss: 0.7090 - val_accuracy: 0.6167
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6320 - accuracy: 0.6333 - val_loss: 0.8200 - val_accuracy: 0.5250
Epoch 8/100


Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 0.0977 - accuracy: 0.9583 - val_loss: 2.3010 - val_accuracy: 0.5250
Epoch 60/100
16/16 [==============================] - 0s 4ms/step - loss: 0.0822 - accuracy: 0.9729 - val_loss: 2.6362 - val_accuracy: 0.5083
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.0987 - accuracy: 0.9646 - val_loss: 2.6024 - val_accuracy: 0.5000
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.0834 - accuracy: 0.9625 - val_loss: 2.6468 - val_accuracy: 0.4667
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.0802 - accuracy: 0.9646 - val_loss: 2.7537 - val_accuracy: 0.5167
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.1092 - accuracy: 0.9646 - val_loss: 2.9537 - val_accuracy: 0.4500
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2249 - accuracy: 0.9271 - val_loss: 2.4949 - val_accuracy: 0.5667
Epoch 

In [135]:
test_pred = model.predict(X_test)
test_pred_class = test_pred.argmax(axis=1)

print(classification_report(y_test, test_pred_class))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75        90
           1       0.00      0.00      0.00        60

    accuracy                           0.60       150
   macro avg       0.30      0.50      0.37       150
weighted avg       0.36      0.60      0.45       150



C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

The model make the simple assumption that everything is normal to yield a 60% score since datasets are unbalanced. Let's try to play with the weight of the classes. 

It also completly overfit the training dataset. Putting weight won't change anything but let's try anyway:

In [138]:
#Create different list of weight
list_dict_weight = []
for weight in np.linspace(2,50,5):
    list_dict_weight.append({
        0:1,
        1: weight
    })
    
#Train and evaluate different model
list_classification_report = []
for dict_weight in list_dict_weight:
    
    #Train model
    model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)
    
    #Predict
    test_pred = model.predict(X_test)
    test_pred_class = test_pred.argmax(axis=1)
    
    #Evaluate and store in list
    list_classification_report.append(classification_report(y_test, test_pred_class))
    
#Report
for class_weights, clf_report in zip (list_dict_weight, list_classification_report):
    print(class_weights)
    print(clf_report)
    print('\n')

Epoch 1/100
16/16 [==============================] - 0s 7ms/step - loss: 3.0693e-07 - accuracy: 1.0000 - val_loss: 9.3085 - val_accuracy: 0.4917
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 3.0375e-07 - accuracy: 1.0000 - val_loss: 9.3147 - val_accuracy: 0.4917
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 3.0303e-07 - accuracy: 1.0000 - val_loss: 9.3197 - val_accuracy: 0.4917
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 2.9740e-07 - accuracy: 1.0000 - val_loss: 9.3277 - val_accuracy: 0.4917
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 2.9596e-07 - accuracy: 1.0000 - val_loss: 9.3369 - val_accuracy: 0.4917
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 2.9259e-07 - accuracy: 1.0000 - val_loss: 9.3395 - val_accuracy: 0.4917
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 2.8940e-07 - accuracy: 1.0000 - val_loss: 9.3440 - val_ac

16/16 [==============================] - 0s 5ms/step - loss: 1.8146e-07 - accuracy: 1.0000 - val_loss: 9.6272 - val_accuracy: 0.4917
Epoch 58/100
16/16 [==============================] - 0s 4ms/step - loss: 1.7940e-07 - accuracy: 1.0000 - val_loss: 9.6336 - val_accuracy: 0.4917
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 1.7792e-07 - accuracy: 1.0000 - val_loss: 9.6378 - val_accuracy: 0.4917
Epoch 60/100
16/16 [==============================] - 0s 4ms/step - loss: 1.7653e-07 - accuracy: 1.0000 - val_loss: 9.6458 - val_accuracy: 0.4917
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 1.7472e-07 - accuracy: 1.0000 - val_loss: 9.6481 - val_accuracy: 0.4917
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 1.7316e-07 - accuracy: 1.0000 - val_loss: 9.6556 - val_accuracy: 0.4917
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 1.7203e-07 - accuracy: 1.0000 - val_loss: 9.6599 - val_accuracy

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 1.1892e-07 - accuracy: 1.0000 - val_loss: 9.8827 - val_accuracy: 0.4917
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 1.1754e-07 - accuracy: 1.0000 - val_loss: 9.8860 - val_accuracy: 0.4917
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 1.1715e-07 - accuracy: 1.0000 - val_loss: 9.8912 - val_accuracy: 0.4917
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 1.1606e-07 - accuracy: 1.0000 - val_loss: 9.9005 - val_accuracy: 0.4917
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 1.1483e-07 - accuracy: 1.0000 - val_loss: 9.9097 - val_accuracy: 0.4917
Epoch 8/100
16/16 [==============================] - 0s 4ms/step - loss: 1.1428e-07 - accuracy: 1.0000 - val_loss: 9.9095 - val_accuracy: 0.4917
Epoch 9/100
16/16 [==============================] - 0s 5ms/step - loss: 1.1229e-07 - accuracy: 1.0000 - val_loss: 9.9166 - val_ac

Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 7.1563e-08 - accuracy: 1.0000 - val_loss: 10.1930 - val_accuracy: 0.4917
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 7.1072e-08 - accuracy: 1.0000 - val_loss: 10.2036 - val_accuracy: 0.4833
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 7.0344e-08 - accuracy: 1.0000 - val_loss: 10.2050 - val_accuracy: 0.4917
Epoch 62/100
16/16 [==============================] - 0s 6ms/step - loss: 6.9755e-08 - accuracy: 1.0000 - val_loss: 10.2146 - val_accuracy: 0.4917
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 6.9089e-08 - accuracy: 1.0000 - val_loss: 10.2143 - val_accuracy: 0.4917
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 6.8494e-08 - accuracy: 1.0000 - val_loss: 10.2261 - val_accuracy: 0.4917
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 6.7811e-08 - accuracy: 1.0000 - val_loss: 10

...

Epoch 58/100
16/16 [==============================] - 0s 5ms/step - loss: 1.4121e-08 - accuracy: 1.0000 - val_loss: 11.1384 - val_accuracy: 0.4750
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 1.4048e-08 - accuracy: 1.0000 - val_loss: 11.1453 - val_accuracy: 0.4750
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 1.3903e-08 - accuracy: 1.0000 - val_loss: 11.1485 - val_accuracy: 0.4750
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 1.3785e-08 - accuracy: 1.0000 - val_loss: 11.1564 - val_accuracy: 0.4750
Epoch 62/100
16/16 [==============================] - 0s 6ms/step - loss: 1.3693e-08 - accuracy: 1.0000 - val_loss: 11.1565 - val_accuracy: 0.4750
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 1.3530e-08 - accuracy: 1.0000 - val_loss: 11.1553 - val_accuracy: 0.4750
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 1.3495e-08 - accuracy: 1.0000 - val_loss: 11

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 1.0824e-08 - accuracy: 1.0000 - val_loss: 11.3111 - val_accuracy: 0.4750
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 1.0731e-08 - accuracy: 1.0000 - val_loss: 11.3130 - val_accuracy: 0.4750
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 1.0664e-08 - accuracy: 1.0000 - val_loss: 11.3162 - val_accuracy: 0.4750
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 1.0576e-08 - accuracy: 1.0000 - val_loss: 11.3230 - val_accuracy: 0.4750
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 1.0593e-08 - accuracy: 1.0000 - val_loss: 11.3254 - val_accuracy: 0.4750
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 1.0523e-08 - accuracy: 1.0000 - val_loss: 11.3290 - val_accuracy: 0.4750
Epoch 8/100
16/16 [==============================] - 0s 6ms/step - loss: 1.0535e-08 - accuracy: 1.0000 - val_loss: 11.3350 - val_accurac

Epoch 58/100
16/16 [==============================] - 0s 5ms/step - loss: 7.8479e-09 - accuracy: 1.0000 - val_loss: 11.5139 - val_accuracy: 0.4833
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 7.8723e-09 - accuracy: 1.0000 - val_loss: 11.5201 - val_accuracy: 0.4833
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 7.8372e-09 - accuracy: 1.0000 - val_loss: 11.5282 - val_accuracy: 0.4833
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 7.7528e-09 - accuracy: 1.0000 - val_loss: 11.5338 - val_accuracy: 0.4833
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 7.7980e-09 - accuracy: 1.0000 - val_loss: 11.5381 - val_accuracy: 0.4833
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 7.7315e-09 - accuracy: 1.0000 - val_loss: 11.5402 - val_accuracy: 0.4833
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 7.6956e-09 - accuracy: 1.0000 - val_loss: 11

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [139]:
#Create different list of weight
list_dict_weight = []
for weight in np.linspace(100,1000,5):
    list_dict_weight.append({
        0:1,
        1: weight
    })
    
#Train and evaluate different model
list_classification_report = []
for dict_weight in list_dict_weight:
    
    #Train model
    model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)
    
    #Predict
    test_pred = model.predict(X_test)
    test_pred_class = test_pred.argmax(axis=1)
    
    #Evaluate and store in list
    list_classification_report.append(classification_report(y_test, test_pred_class))
    
#Report
for class_weights, clf_report in zip (list_dict_weight, list_classification_report):
    print(class_weights)
    print(clf_report)
    print('\n')

Epoch 1/100
16/16 [==============================] - 0s 6ms/step - loss: 6.5004e-09 - accuracy: 1.0000 - val_loss: 11.6627 - val_accuracy: 0.4833
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 6.4623e-09 - accuracy: 1.0000 - val_loss: 11.6697 - val_accuracy: 0.4833
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 6.4130e-09 - accuracy: 1.0000 - val_loss: 11.6713 - val_accuracy: 0.4833
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 6.3774e-09 - accuracy: 1.0000 - val_loss: 11.6788 - val_accuracy: 0.4833
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 6.3543e-09 - accuracy: 1.0000 - val_loss: 11.6785 - val_accuracy: 0.4833
Epoch 6/100
16/16 [==============================] - 0s 4ms/step - loss: 6.4767e-09 - accuracy: 1.0000 - val_loss: 11.6842 - val_accuracy: 0.4833
Epoch 7/100
16/16 [==============================] - 0s 4ms/step - loss: 6.3785e-09 - accuracy: 1.0000 - val_loss: 11.6892 -

Epoch 57/100
16/16 [==============================] - 0s 5ms/step - loss: 5.6396e-09 - accuracy: 1.0000 - val_loss: 11.8231 - val_accuracy: 0.4833
Epoch 58/100
16/16 [==============================] - 0s 5ms/step - loss: 5.4613e-09 - accuracy: 1.0000 - val_loss: 11.8274 - val_accuracy: 0.4833
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 5.3308e-09 - accuracy: 1.0000 - val_loss: 11.8243 - val_accuracy: 0.4833
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 5.3320e-09 - accuracy: 1.0000 - val_loss: 11.8264 - val_accuracy: 0.4833
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 5.4026e-09 - accuracy: 1.0000 - val_loss: 11.8265 - val_accuracy: 0.4833
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 5.3939e-09 - accuracy: 1.0000 - val_loss: 11.8385 - val_accuracy: 0.4833
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 5.4096e-09 - accuracy: 1.0000 - val_loss: 11

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 5.0494e-09 - accuracy: 1.0000 - val_loss: 11.9231 - val_accuracy: 0.4833
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 5.1239e-09 - accuracy: 1.0000 - val_loss: 11.9203 - val_accuracy: 0.4833
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 5.0901e-09 - accuracy: 1.0000 - val_loss: 11.9174 - val_accuracy: 0.4833
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 5.1845e-09 - accuracy: 1.0000 - val_loss: 11.9145 - val_accuracy: 0.4833
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 5.0539e-09 - accuracy: 1.0000 - val_loss: 11.9315 - val_accuracy: 0.4833
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 5.0676e-09 - accuracy: 1.0000 - val_loss: 11.9367 - val_accuracy: 0.4833
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 4.9917e-09 - accuracy: 1.0000 - val_loss: 11.9352 - val_accurac

Epoch 58/100
16/16 [==============================] - 0s 5ms/step - loss: 4.9899e-09 - accuracy: 1.0000 - val_loss: 12.0314 - val_accuracy: 0.4917
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 5.0005e-09 - accuracy: 1.0000 - val_loss: 12.0322 - val_accuracy: 0.4917
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 4.8954e-09 - accuracy: 1.0000 - val_loss: 12.0247 - val_accuracy: 0.4917
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 5.0547e-09 - accuracy: 1.0000 - val_loss: 12.0349 - val_accuracy: 0.4833
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 4.5631e-09 - accuracy: 1.0000 - val_loss: 12.0514 - val_accuracy: 0.4833
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 4.6568e-09 - accuracy: 1.0000 - val_loss: 12.0502 - val_accuracy: 0.4833
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 4.6307e-09 - accuracy: 1.0000 - val_loss: 12

...

Epoch 58/100
16/16 [==============================] - 0s 5ms/step - loss: 4.2088e-09 - accuracy: 1.0000 - val_loss: 12.2652 - val_accuracy: 0.5000
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 3.4787e-09 - accuracy: 1.0000 - val_loss: 12.2459 - val_accuracy: 0.5000
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 3.5239e-09 - accuracy: 1.0000 - val_loss: 12.2442 - val_accuracy: 0.5000
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 3.7514e-09 - accuracy: 1.0000 - val_loss: 12.2492 - val_accuracy: 0.5000
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 3.7196e-09 - accuracy: 1.0000 - val_loss: 12.2762 - val_accuracy: 0.4917
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 3.6104e-09 - accuracy: 1.0000 - val_loss: 12.2821 - val_accuracy: 0.4917
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 3.7437e-09 - accuracy: 1.0000 - val_loss: 12

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 3.6508e-09 - accuracy: 1.0000 - val_loss: 12.3840 - val_accuracy: 0.4917
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 3.6701e-09 - accuracy: 1.0000 - val_loss: 12.3860 - val_accuracy: 0.5000
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 3.9114e-09 - accuracy: 1.0000 - val_loss: 12.3845 - val_accuracy: 0.5000
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 3.9067e-09 - accuracy: 1.0000 - val_loss: 12.4007 - val_accuracy: 0.5000
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 3.3122e-09 - accuracy: 1.0000 - val_loss: 12.4068 - val_accuracy: 0.5000
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 3.3789e-09 - accuracy: 1.0000 - val_loss: 12.4031 - val_accuracy: 0.5000
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 3.6361e-09 - accuracy: 1.0000 - val_loss: 12.3986 - val_accurac

Epoch 58/100
16/16 [==============================] - 0s 5ms/step - loss: 2.4977e-09 - accuracy: 1.0000 - val_loss: 12.4016 - val_accuracy: 0.4917
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 2.6297e-09 - accuracy: 1.0000 - val_loss: 12.4037 - val_accuracy: 0.4917
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 2.8764e-09 - accuracy: 1.0000 - val_loss: 12.4025 - val_accuracy: 0.4917
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 3.0836e-09 - accuracy: 1.0000 - val_loss: 12.3996 - val_accuracy: 0.4917
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 3.2750e-09 - accuracy: 1.0000 - val_loss: 12.4031 - val_accuracy: 0.4917
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 2.8256e-09 - accuracy: 1.0000 - val_loss: 12.4070 - val_accuracy: 0.4917
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 2.8491e-09 - accuracy: 1.0000 - val_loss: 12

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

On peut le constater, aucun modèle n'arrive à détecter les anomalies. Les points sur lesquels on pourrait apporter de grandes améliorations sont:

    1. Utilisation d'une plus grande résolution de données
    2. Utilisation de modèles plus complexes 
    
Un exemple de modèle plus complexe peut être l'utilisation tout d'abord d'un ANN afin de classifier les différents types de machines. Cette tâche sera la tâche auxilliaire. Les données condensées d'une des couches de cet ANN sera ensuite utilisé pour la classification anomalie/normal avec différents modèles.

Let's just try with Dropout layers

In [142]:
#Initialization of the layers
inputs = Input(shape=X_train.shape[1], name='Inputs')
first_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='First_layer')
first_dropout = Dropout(0.4, name='First_dropout')
second_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='Second_layer')
second_dropout = Dropout(0.4, name='Second_dropout')
third_layer = Dense(64, activation='relu', kernel_initializer ='normal', name='Third_layer')
third_dropout = Dropout(0.4, name='Third_dropout')
fourth_layer = Dense(1, activation='sigmoid', kernel_initializer ='normal', name='Output_layer')

#Construct the layer's order
x = first_layer(inputs)
x = first_dropout(x)
x = second_layer(x)
x = second_dropout(x)
x = third_layer(x)
x = third_dropout(x)
outputs = fourth_layer(x)

#Initialize the model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Inputs (InputLayer)         [(None, 1024)]            0         
                                                                 
 First_layer (Dense)         (None, 128)               131200    
                                                                 
 First_dropout (Dropout)     (None, 128)               0         
                                                                 
 Second_layer (Dense)        (None, 128)               16512     
                                                                 
 Second_dropout (Dropout)    (None, 128)               0         
                                                                 
 Third_layer (Dense)         (None, 64)                8256      
                                                                 
 Third_dropout (Dropout)     (None, 64)                0   

In [143]:
#Create different list of weight
list_dict_weight = []
for weight in np.linspace(1,50,5):
    list_dict_weight.append({
        0:1,
        1: weight
    })
    
#Train and evaluate different model
list_classification_report = []
for dict_weight in list_dict_weight:
    
    #Train model
    model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)
    
    #Predict
    test_pred = model.predict(X_test)
    test_pred_class = test_pred.argmax(axis=1)
    
    #Evaluate and store in list
    list_classification_report.append(classification_report(y_test, test_pred_class))
    
#Report
for class_weights, clf_report in zip (list_dict_weight, list_classification_report):
    print(class_weights)
    print(clf_report)
    print('\n')

Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.6989 - accuracy: 0.5292 - val_loss: 0.6902 - val_accuracy: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6750 - accuracy: 0.5917 - val_loss: 0.7153 - val_accuracy: 0.6167
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6765 - accuracy: 0.5938 - val_loss: 0.7130 - val_accuracy: 0.5667
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6966 - accuracy: 0.5646 - val_loss: 0.6940 - val_accuracy: 0.6083
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6848 - accuracy: 0.5750 - val_loss: 0.6918 - val_accuracy: 0.5917
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6770 - accuracy: 0.5750 - val_loss: 0.7144 - val_accuracy: 0.5500
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6699 - accuracy: 0.6042 - val_loss: 0.7186 - val_accuracy: 0.5333
Epoch 8/100


Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5454 - accuracy: 0.6854 - val_loss: 0.8331 - val_accuracy: 0.5500
Epoch 60/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5408 - accuracy: 0.7063 - val_loss: 0.7780 - val_accuracy: 0.5583
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5785 - accuracy: 0.6812 - val_loss: 0.7693 - val_accuracy: 0.5750
Epoch 62/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5526 - accuracy: 0.7104 - val_loss: 0.7622 - val_accuracy: 0.5583
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5627 - accuracy: 0.6875 - val_loss: 0.7723 - val_accuracy: 0.5333
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5589 - accuracy: 0.7146 - val_loss: 0.7861 - val_accuracy: 0.5500
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5339 - accuracy: 0.7021 - val_loss: 0.7749 - val_accuracy: 0.5833
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 0.4606 - accuracy: 0.7396 - val_loss: 1.0159 - val_accuracy: 0.5750
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.4707 - accuracy: 0.7312 - val_loss: 1.1132 - val_accuracy: 0.5583
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 0.4535 - accuracy: 0.7479 - val_loss: 1.0328 - val_accuracy: 0.5750
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4653 - accuracy: 0.7563 - val_loss: 0.9191 - val_accuracy: 0.6083
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4843 - accuracy: 0.7437 - val_loss: 1.0502 - val_accuracy: 0.5333
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4842 - accuracy: 0.7083 - val_loss: 1.0672 - val_accuracy: 0.5167
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4767 - accuracy: 0.7500 - val_loss: 1.0674 - val_accuracy: 0.5750
Epoch 9/100
16/16 [======

Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 0.3901 - accuracy: 0.7833 - val_loss: 1.5038 - val_accuracy: 0.5250
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 0.4060 - accuracy: 0.7833 - val_loss: 1.4105 - val_accuracy: 0.5417
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4123 - accuracy: 0.7875 - val_loss: 1.3023 - val_accuracy: 0.5417
Epoch 63/100
16/16 [==============================] - 0s 6ms/step - loss: 0.3980 - accuracy: 0.7667 - val_loss: 1.2738 - val_accuracy: 0.5417
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4021 - accuracy: 0.7708 - val_loss: 1.4450 - val_accuracy: 0.5250
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.3947 - accuracy: 0.7979 - val_loss: 1.4343 - val_accuracy: 0.5167
Epoch 66/100
16/16 [==============================] - 0s 5ms/step - loss: 0.3985 - accuracy: 0.7979 - val_loss: 1.3693 - val_accuracy: 0.5667
Epoch 

...

Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2647 - accuracy: 0.8562 - val_loss: 1.9571 - val_accuracy: 0.5167
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2864 - accuracy: 0.8729 - val_loss: 1.7213 - val_accuracy: 0.4917
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2748 - accuracy: 0.8625 - val_loss: 1.8164 - val_accuracy: 0.5167
Epoch 63/100
16/16 [==============================] - 0s 6ms/step - loss: 0.2719 - accuracy: 0.8813 - val_loss: 1.8107 - val_accuracy: 0.5083
Epoch 64/100
16/16 [==============================] - 0s 6ms/step - loss: 0.2748 - accuracy: 0.8604 - val_loss: 1.7678 - val_accuracy: 0.5083
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.2439 - accuracy: 0.9042 - val_loss: 1.9170 - val_accuracy: 0.5083
Epoch 66/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2781 - accuracy: 0.8771 - val_loss: 1.6862 - val_accuracy: 0.5333
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 0.2461 - accuracy: 0.8875 - val_loss: 2.0364 - val_accuracy: 0.5250
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2167 - accuracy: 0.8917 - val_loss: 2.0218 - val_accuracy: 0.5333
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2766 - accuracy: 0.8917 - val_loss: 1.9055 - val_accuracy: 0.5083
Epoch 5/100
16/16 [==============================] - 0s 7ms/step - loss: 0.2655 - accuracy: 0.8854 - val_loss: 1.8627 - val_accuracy: 0.4833
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2353 - accuracy: 0.8729 - val_loss: 1.7820 - val_accuracy: 0.5083
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2692 - accuracy: 0.8875 - val_loss: 1.7955 - val_accuracy: 0.5167
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2606 - accuracy: 0.8708 - val_loss: 1.9351 - val_accuracy: 0.5000
Epoch 9/100
16/16 [======

Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2782 - accuracy: 0.8625 - val_loss: 1.9823 - val_accuracy: 0.5583
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2694 - accuracy: 0.8833 - val_loss: 1.7878 - val_accuracy: 0.5417
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2248 - accuracy: 0.8917 - val_loss: 1.8725 - val_accuracy: 0.5167
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2013 - accuracy: 0.9021 - val_loss: 2.1084 - val_accuracy: 0.5417
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2461 - accuracy: 0.8792 - val_loss: 2.2675 - val_accuracy: 0.5333
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.2447 - accuracy: 0.8938 - val_loss: 1.9865 - val_accuracy: 0.5583
Epoch 66/100
16/16 [==============================] - 0s 6ms/step - loss: 0.2091 - accuracy: 0.9000 - val_loss: 2.0397 - val_accuracy: 0.4917
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [144]:
#Initializaiton of the layers
inputs = Input(shape=X_train.shape[1], name='Inputs')
first_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='First_layer')
first_dropout = Dropout(0.7, name='First_dropout')
second_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='Second_layer')
second_dropout = Dropout(0.6, name='Second_dropout')
third_layer = Dense(64, activation='relu', kernel_initializer ='normal', name='Third_layer')
third_dropout = Dropout(0.5, name='Third_dropout')
fourth_layer = Dense(1, activation='sigmoid', kernel_initializer ='normal', name='Output_layer')

#Construct the layer's order
x = first_layer(inputs)
x = first_dropout(x)
x = second_layer(x)
x = second_dropout(x)
x = third_layer(x)
x = third_dropout(x)
outputs = fourth_layer(x)

#Initialize the model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Inputs (InputLayer)         [(None, 1024)]            0         
                                                                 
 First_layer (Dense)         (None, 128)               131200    
                                                                 
 First_dropout (Dropout)     (None, 128)               0         
                                                                 
 Second_layer (Dense)        (None, 128)               16512     
                                                                 
 Second_dropout (Dropout)    (None, 128)               0         
                                                                 
 Third_layer (Dense)         (None, 64)                8256      
                                                                 
 Third_dropout (Dropout)     (None, 64)                0   

In [145]:
#Create different list of weight
list_dict_weight = []
for weight in np.linspace(1,50,5):
    list_dict_weight.append({
        0:1,
        1: weight
    })
    
#Train and evaluate different model
list_classification_report = []
for dict_weight in list_dict_weight:
    
    #Train model
    model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)
    
    #Predict
    test_pred = model.predict(X_test)
    test_pred_class = test_pred.argmax(axis=1)
    
    #Evaluate and store in list
    list_classification_report.append(classification_report(y_test, test_pred_class))
    
#Report
for class_weights, clf_report in zip (list_dict_weight, list_classification_report):
    print(class_weights)
    print(clf_report)
    print('\n')

Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.6912 - accuracy: 0.5646 - val_loss: 0.6895 - val_accuracy: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.7024 - accuracy: 0.5500 - val_loss: 0.6843 - val_accuracy: 0.6250
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7221 - accuracy: 0.5521 - val_loss: 0.6920 - val_accuracy: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6910 - accuracy: 0.5479 - val_loss: 0.6879 - val_accuracy: 0.6250
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6987 - accuracy: 0.5625 - val_loss: 0.6860 - val_accuracy: 0.6250
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7013 - accuracy: 0.5396 - val_loss: 0.6920 - val_accuracy: 0.6167
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6889 - accuracy: 0.5688 - val_loss: 0.6904 - val_accuracy: 0.6250
Epoch 8/100


Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6599 - accuracy: 0.6042 - val_loss: 0.6844 - val_accuracy: 0.6000
Epoch 60/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6510 - accuracy: 0.6167 - val_loss: 0.6946 - val_accuracy: 0.5333
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6616 - accuracy: 0.6187 - val_loss: 0.6953 - val_accuracy: 0.5417
Epoch 62/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6666 - accuracy: 0.6125 - val_loss: 0.6923 - val_accuracy: 0.5417
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6580 - accuracy: 0.6000 - val_loss: 0.6823 - val_accuracy: 0.6083
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6510 - accuracy: 0.6187 - val_loss: 0.6832 - val_accuracy: 0.6250
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6588 - accuracy: 0.5958 - val_loss: 0.6839 - val_accuracy: 0.6083
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 0.6494 - accuracy: 0.6167 - val_loss: 0.6856 - val_accuracy: 0.6083
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6381 - accuracy: 0.6208 - val_loss: 0.6792 - val_accuracy: 0.6083
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6548 - accuracy: 0.5833 - val_loss: 0.6686 - val_accuracy: 0.6167
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6641 - accuracy: 0.5875 - val_loss: 0.6688 - val_accuracy: 0.6500
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6337 - accuracy: 0.6104 - val_loss: 0.6792 - val_accuracy: 0.6333
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6453 - accuracy: 0.6104 - val_loss: 0.6860 - val_accuracy: 0.6333
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6309 - accuracy: 0.6146 - val_loss: 0.6740 - val_accuracy: 0.6500
Epoch 9/100
16/16 [======

Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6005 - accuracy: 0.6833 - val_loss: 0.6927 - val_accuracy: 0.6417
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5967 - accuracy: 0.6625 - val_loss: 0.7151 - val_accuracy: 0.6250
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6181 - accuracy: 0.6687 - val_loss: 0.6890 - val_accuracy: 0.6333
Epoch 63/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6098 - accuracy: 0.6687 - val_loss: 0.7012 - val_accuracy: 0.6000
Epoch 64/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6133 - accuracy: 0.6438 - val_loss: 0.6933 - val_accuracy: 0.6500
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5934 - accuracy: 0.6646 - val_loss: 0.7021 - val_accuracy: 0.6250
Epoch 66/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6176 - accuracy: 0.6396 - val_loss: 0.6975 - val_accuracy: 0.6333
Epoch 

...

Epoch 60/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5417 - accuracy: 0.7104 - val_loss: 0.8413 - val_accuracy: 0.6333
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5145 - accuracy: 0.7104 - val_loss: 0.8642 - val_accuracy: 0.6250
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5011 - accuracy: 0.7063 - val_loss: 0.9755 - val_accuracy: 0.6000
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5476 - accuracy: 0.7042 - val_loss: 0.9056 - val_accuracy: 0.6250
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4983 - accuracy: 0.7167 - val_loss: 0.9316 - val_accuracy: 0.6417
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5080 - accuracy: 0.7000 - val_loss: 0.9433 - val_accuracy: 0.6417
Epoch 66/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4863 - accuracy: 0.7042 - val_loss: 0.9524 - val_accuracy: 0.6250
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 0.5010 - accuracy: 0.6917 - val_loss: 0.9704 - val_accuracy: 0.6167
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5119 - accuracy: 0.7333 - val_loss: 0.9090 - val_accuracy: 0.6167
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5157 - accuracy: 0.7229 - val_loss: 0.8985 - val_accuracy: 0.6167
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4831 - accuracy: 0.7292 - val_loss: 0.9714 - val_accuracy: 0.6000
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5244 - accuracy: 0.6979 - val_loss: 0.9119 - val_accuracy: 0.6250
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5164 - accuracy: 0.7000 - val_loss: 0.9297 - val_accuracy: 0.6250
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4847 - accuracy: 0.7125 - val_loss: 1.0036 - val_accuracy: 0.6167
Epoch 9/100
16/16 [======

Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5009 - accuracy: 0.7188 - val_loss: 0.9860 - val_accuracy: 0.6167
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4786 - accuracy: 0.7167 - val_loss: 0.9917 - val_accuracy: 0.6333
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4698 - accuracy: 0.7479 - val_loss: 1.0661 - val_accuracy: 0.6250
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4628 - accuracy: 0.7188 - val_loss: 1.0846 - val_accuracy: 0.6250
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4988 - accuracy: 0.7000 - val_loss: 1.0774 - val_accuracy: 0.6167
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5270 - accuracy: 0.7000 - val_loss: 0.9559 - val_accuracy: 0.6250
Epoch 66/100
16/16 [==============================] - 0s 6ms/step - loss: 0.4983 - accuracy: 0.7125 - val_loss: 0.8998 - val_accuracy: 0.6250
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [146]:
#Initializaiton of the layers
inputs = Input(shape=X_train.shape[1], name='Inputs')
first_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='First_layer')
first_dropout = Dropout(0.8, name='First_dropout')
second_layer = Dense(128, activation='relu', kernel_initializer ='normal', name='Second_layer')
second_dropout = Dropout(0.7, name='Second_dropout')
third_layer = Dense(64, activation='relu', kernel_initializer ='normal', name='Third_layer')
third_dropout = Dropout(0.7, name='Third_dropout')
fourth_layer = Dense(1, activation='sigmoid', kernel_initializer ='normal', name='Output_layer')

#Construct the layer's order
x = first_layer(inputs)
x = first_dropout(x)
x = second_layer(x)
x = second_dropout(x)
x = third_layer(x)
x = third_dropout(x)
outputs = fourth_layer(x)

#Initialize the model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Inputs (InputLayer)         [(None, 1024)]            0         
                                                                 
 First_layer (Dense)         (None, 128)               131200    
                                                                 
 First_dropout (Dropout)     (None, 128)               0         
                                                                 
 Second_layer (Dense)        (None, 128)               16512     
                                                                 
 Second_dropout (Dropout)    (None, 128)               0         
                                                                 
 Third_layer (Dense)         (None, 64)                8256      
                                                                 
 Third_dropout (Dropout)     (None, 64)                0   

In [147]:
#Create different list of weight
list_dict_weight = []
for weight in np.linspace(1,50,5):
    list_dict_weight.append({
        0:1,
        1: weight
    })
    
#Train and evaluate different model
list_classification_report = []
for dict_weight in list_dict_weight:
    
    #Train model
    model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)
    
    #Predict
    test_pred = model.predict(X_test)
    test_pred_class = test_pred.argmax(axis=1)
    
    #Evaluate and store in list
    list_classification_report.append(classification_report(y_test, test_pred_class))
    
#Report
for class_weights, clf_report in zip (list_dict_weight, list_classification_report):
    print(class_weights)
    print(clf_report)
    print('\n')

Epoch 1/100
16/16 [==============================] - 0s 11ms/step - loss: 0.7267 - accuracy: 0.5458 - val_loss: 0.6817 - val_accuracy: 0.6167
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7650 - accuracy: 0.5417 - val_loss: 0.6979 - val_accuracy: 0.5917
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7264 - accuracy: 0.5458 - val_loss: 0.6985 - val_accuracy: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7184 - accuracy: 0.5354 - val_loss: 0.6970 - val_accuracy: 0.6250
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 0.7159 - accuracy: 0.5396 - val_loss: 0.6937 - val_accuracy: 0.6250
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7136 - accuracy: 0.5375 - val_loss: 0.6921 - val_accuracy: 0.6250
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.7068 - accuracy: 0.5667 - val_loss: 0.6886 - val_accuracy: 0.6250
Epoch 8/100


Epoch 59/100
16/16 [==============================] - 0s 8ms/step - loss: 0.6789 - accuracy: 0.5833 - val_loss: 0.6755 - val_accuracy: 0.6250
Epoch 60/100
16/16 [==============================] - 0s 7ms/step - loss: 0.6784 - accuracy: 0.5875 - val_loss: 0.6748 - val_accuracy: 0.6250
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6792 - accuracy: 0.5938 - val_loss: 0.6746 - val_accuracy: 0.6250
Epoch 62/100
16/16 [==============================] - 0s 8ms/step - loss: 0.6721 - accuracy: 0.5896 - val_loss: 0.6743 - val_accuracy: 0.6250
Epoch 63/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6773 - accuracy: 0.5917 - val_loss: 0.6742 - val_accuracy: 0.6250
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6807 - accuracy: 0.5854 - val_loss: 0.6737 - val_accuracy: 0.6250
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6790 - accuracy: 0.5917 - val_loss: 0.6736 - val_accuracy: 0.6250
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 0.6721 - accuracy: 0.5875 - val_loss: 0.6766 - val_accuracy: 0.6250
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6911 - accuracy: 0.5792 - val_loss: 0.6737 - val_accuracy: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6725 - accuracy: 0.5896 - val_loss: 0.6718 - val_accuracy: 0.6250
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6777 - accuracy: 0.5979 - val_loss: 0.6724 - val_accuracy: 0.6250
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6772 - accuracy: 0.5938 - val_loss: 0.6713 - val_accuracy: 0.6250
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6617 - accuracy: 0.5979 - val_loss: 0.6721 - val_accuracy: 0.6250
Epoch 8/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6788 - accuracy: 0.5875 - val_loss: 0.6714 - val_accuracy: 0.6250
Epoch 9/100
16/16 [======

Epoch 60/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6723 - accuracy: 0.5917 - val_loss: 0.6661 - val_accuracy: 0.6250
Epoch 61/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6772 - accuracy: 0.5917 - val_loss: 0.6676 - val_accuracy: 0.6250
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6633 - accuracy: 0.5917 - val_loss: 0.6680 - val_accuracy: 0.6250
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6751 - accuracy: 0.5938 - val_loss: 0.6702 - val_accuracy: 0.6250
Epoch 64/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6700 - accuracy: 0.5917 - val_loss: 0.6706 - val_accuracy: 0.6250
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6630 - accuracy: 0.5833 - val_loss: 0.6700 - val_accuracy: 0.6250
Epoch 66/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6858 - accuracy: 0.5875 - val_loss: 0.6683 - val_accuracy: 0.6250
Epoch 

...

Epoch 60/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6443 - accuracy: 0.5917 - val_loss: 0.6890 - val_accuracy: 0.6250
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6445 - accuracy: 0.5938 - val_loss: 0.6916 - val_accuracy: 0.6250
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6482 - accuracy: 0.5854 - val_loss: 0.6820 - val_accuracy: 0.6250
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6520 - accuracy: 0.5958 - val_loss: 0.6767 - val_accuracy: 0.6250
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6425 - accuracy: 0.5833 - val_loss: 0.6866 - val_accuracy: 0.6250
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6702 - accuracy: 0.5854 - val_loss: 0.6758 - val_accuracy: 0.6250
Epoch 66/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6551 - accuracy: 0.5917 - val_loss: 0.6748 - val_accuracy: 0.6250
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

16/16 [==============================] - 0s 5ms/step - loss: 0.6597 - accuracy: 0.5792 - val_loss: 0.6707 - val_accuracy: 0.6250
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6673 - accuracy: 0.5917 - val_loss: 0.6719 - val_accuracy: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6440 - accuracy: 0.5896 - val_loss: 0.6789 - val_accuracy: 0.6250
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6388 - accuracy: 0.5979 - val_loss: 0.6840 - val_accuracy: 0.6250
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6540 - accuracy: 0.5917 - val_loss: 0.6797 - val_accuracy: 0.6250
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6413 - accuracy: 0.5917 - val_loss: 0.6754 - val_accuracy: 0.6250
Epoch 8/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6757 - accuracy: 0.5958 - val_loss: 0.6731 - val_accuracy: 0.6250
Epoch 9/100
16/16 [======

Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6247 - accuracy: 0.5958 - val_loss: 0.6825 - val_accuracy: 0.6250
Epoch 61/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6300 - accuracy: 0.5938 - val_loss: 0.6960 - val_accuracy: 0.6250
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6319 - accuracy: 0.5979 - val_loss: 0.6961 - val_accuracy: 0.6250
Epoch 63/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6367 - accuracy: 0.5875 - val_loss: 0.6916 - val_accuracy: 0.6250
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6642 - accuracy: 0.5917 - val_loss: 0.6801 - val_accuracy: 0.6250
Epoch 65/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6372 - accuracy: 0.5917 - val_loss: 0.6761 - val_accuracy: 0.6250
Epoch 66/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6548 - accuracy: 0.5896 - val_loss: 0.6843 - val_accuracy: 0.6250
Epoch 

C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta